In [1]:
from aocd import get_data, submit
# from aocd.get import current_day

from IPython.display import display
from IPython.core.display import Markdown

import math
import numpy as np
import re
import sys

In [2]:
DAY = 19
YEAR = 2023

In [3]:
url = f"https://adventofcode.com/{YEAR}/day/{DAY}"
display(Markdown(f"#### See [{YEAR} Day {DAY}]({url})."))

#### See [2023 Day 19](https://adventofcode.com/2023/day/19).

In [4]:
data = get_data(year=YEAR, day=DAY)

In [5]:
data.splitlines()[:5]

['bvh{a>2239:lr,m>657:mqd,x>928:lz,ndn}',
 'lvc{m<3522:R,m>3836:R,A}',
 'tg{x<1026:R,R}',
 'cdj{m<3527:R,x<3765:A,R}',
 'fgz{a>3380:A,a>2932:A,A}']

# Part A

In [6]:
# data = """px{a<2006:qkq,m>2090:A,rfg}
# pv{a>1716:R,A}
# lnx{m>1548:A,A}
# rfg{s<537:gd,x>2440:R,A}
# qs{s>3448:A,lnx}
# qkq{x<1416:A,crn}
# crn{x>2662:A,R}
# in{s<1351:px,qqz}
# qqz{s>2770:qs,m<1801:hdj,R}
# gd{a>3333:R,R}
# hdj{m>838:A,pv}

# {x=787,m=2655,a=1222,s=2876}
# {x=1679,m=44,a=2067,s=496}
# {x=2036,m=264,a=79,s=2244}
# {x=2461,m=1339,a=466,s=291}
# {x=2127,m=1623,a=2188,s=1013}
# """

In [7]:
rules, parts = data.split('\n\n')

rules_raw = rules.splitlines()
parts_raw = parts.splitlines()

In [8]:
workflows = {}
for r in rules_raw:
    label, cond = r.split('{')
    cond = cond[:-1]
    conds = cond.split(',')
    cc = []
    for c in conds:
        if '<' in c:
            var, thresh, n = re.split('<|:',c)
            cc.append({'type': '<', 'var': var, 'thresh': int(thresh), 'next': n})
        elif '>' in c:
            var, thresh, n = re.split('>|:',c)
            cc.append({'type': '>', 'var': var, 'thresh': int(thresh), 'next': n})
        else:
            cc.append({'type': 'else', 'next': c})        
    workflows[label] = cc

In [9]:
Q = []
for p in parts_raw:
    n = tuple(map(int, re.findall(r'\d+',p)))
    n = {'x':n[0],'m':n[1],'a':n[2],'s':n[3]}
    Q.append((n,'in'))

accepted = []
rejected = []
while Q:
    p,r = Q.pop()
    workflow = workflows[r]
    for rule in workflow:
        match rule['type']:
            case '<':
                if p[rule['var']] < rule['thresh']:
                    next = rule['next']
                    break
            case '>':
                if p[rule['var']] > rule['thresh']:
                    next = rule['next']
                    break
            case 'else':
                next = rule['next']
    match next:
        case 'A':
            accepted.append(p)
        case 'R':
            rejected.append(p)
        case _:
            Q.append((p,next))

In [10]:
total = 0
for a in accepted:
    total += sum(a.values())
total

373302

In [55]:
submit(total, year=YEAR, day=DAY)

answer a: None
submitting for part a


That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


# Part B

In [11]:
import portion as P

In [12]:
MIN = 1
MAX = 4000

start = {'x': P.closed(MIN,MAX), 
         'm': P.closed(MIN,MAX), 
         'a': P.closed(MIN,MAX), 
         's': P.closed(MIN,MAX)}

Q = [(start, 'in')]

accepted = []
rejected = []
while Q:
    p,r = Q.pop()

    if r == 'A':
        accepted.append(p)
        continue
    elif r == 'R':
        rejected.append(r)
        continue

    workflow = workflows[r]
    for rule in workflow:
        match rule['type']:
            case '<':
                matched = p.copy()
                matched[rule['var']] &= P.closed(MIN,rule['thresh']-1)
                Q.append((matched, rule['next']))
                p[rule['var']] -= P.closedopen(MIN,rule['thresh'])
            case '>':
                matched = p.copy()
                matched[rule['var']] &= P.closed(rule['thresh']+1, MAX)
                Q.append((matched, rule['next']))
                p[rule['var']] -= P.openclosed(rule['thresh'], MAX)
            case 'else':
                Q.append((p,rule['next']))


In [13]:
total = 0
for a in accepted:
    p = 1
    for r in a.values():
        p *= r.upper - r.lower + 1
    total += p

In [14]:
total

130262715574114

In [74]:
submit(total, year=YEAR, day=DAY)

answer a: 373302
submitting for part b (part a is already completed)


That's the right answer!  You are one gold star closer to restoring snow operations. You got rank 761 on this star's leaderboard.You have completed Day 19! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
